In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
path = r"/content/drive/My Drive/Colab Notebooks/DataHack - Analytics Vidhya /JantaHack: Machine Learning in Agriculture"
train = pd.read_csv(path + "/train.csv")
test = pd.read_csv(path + "/test.csv")
submission = pd.read_csv(path + "/sample_submission.csv")


In [5]:
train.shape, test.shape, submission.shape

((88858, 10), (59310, 9), (59310, 2))

**Filling the null values in Number_Weeks_Used column**

In [6]:
train['Number_Weeks_Used'] = train['Number_Weeks_Used'].fillna(train.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].transform('median'))
test['Number_Weeks_Used'] = test['Number_Weeks_Used'].fillna(test.groupby('Pesticide_Use_Category')['Number_Weeks_Used'].transform('median'))

# Data Preprocessing

In [7]:
training_labels = train.iloc[:, -1]
X_train = train.iloc[:, 1:-1]
X_test = test.iloc[:, 1:]

In [8]:
X_train.shape, X_test.shape

((88858, 8), (59310, 8))

In [9]:
data = pd.concat([X_train, X_test])

**OneHotEncoding for "Pesticide_Use_Category" and "Season"**

In [10]:
columns_names_encod = data.columns[[3, 7]]
data = pd.get_dummies(data, columns=columns_names_encod)

In [11]:
data.shape

(148168, 12)

**Splitting the data back**

In [12]:
final_train = data[:train.shape[0]]
final_test = data[train.shape[0]:]

In [13]:
final_train.shape, final_test.shape

((88858, 12), (59310, 12))

# Creating cross validation set

In [14]:
training_labels.value_counts()

0    74238
1    12307
2     2313
Name: Crop_Damage, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
X_training, X_cv, y_training, y_cv = train_test_split(final_train, training_labels, test_size =0.2, random_state=21)

# Training model on training dataset


In [16]:
# To check whether GPU is running or not
import tensorflow as tf
tf.test.gpu_device_name()


'/device:GPU:0'

In [17]:
# lightgbm for classification
from numpy import mean
from numpy import std
#from sklearn.datasets import make_classification
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
#from matplotlib import pyplot
# define dataset
#X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=1)

# evaluate the model
model = LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_training, y_training, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

# fit the model on the whole dataset
model = LGBMClassifier()
model.fit(X_training, y_training)

y_training_pred = model.predict(X_training)
y_cv_pred = model.predict(X_cv)

Accuracy: 0.847 (0.002)


**Checking accuracy score for training dataset and cross validation set**

In [18]:
from sklearn.metrics import accuracy_score
training_accuracy_score = accuracy_score(y_training, y_training_pred)
cv_accuracy_score = accuracy_score(y_cv, y_cv_pred)

print(training_accuracy_score)
print(cv_accuracy_score)

0.851109923191627
0.8445307224848075


# Testing the model on test data

In [19]:
predictions_test = model.predict(final_test)
predictions_test = pd.Series(predictions_test)

frame = {'ID': test.ID, 
         'Crop_Damage': predictions_test}
submission = pd.DataFrame(frame)
submission.to_csv(path + '/lgbm_submission.csv', index=False)